In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-26"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9368,2024-03-26,Eua Nba,21:00,New Orleans Pelicans,Oklahoma City Thunder,2.04,1.66,227.5,1.85,1.95,1.5,1.94,1.73,tpXYrdfM,0.490196,0.602410,0.540541,0.512821,0.092606,0.4,0.2,NaN,NaN,175.970,20.052378,0.113953,182.304,29.524804,0.161954,163.698,194.396,194.79,200.88,0.0,0.0,0.0,0.0,0.145244,0.037216,0.080922,-1.19,-0.238,-4.369748,0.691859,0.6,-0.091859,-1.76,-0.352,-1.875000,0.598619,0.5,-0.098619
9369,2024-03-26,Argentina Liga A,21:00,Comunicaciones Mercedes,San Martin,1.78,1.94,157.5,1.82,1.88,-2.5,2.00,2.30,KxFO1UWt,0.561798,0.515464,0.549451,0.531915,0.077262,0.4,0.6,NaN,NaN,184.672,84.735954,0.458846,133.946,49.572885,0.370096,184.672,176.332,131.72,207.92,0.0,0.0,0.0,0.0,0.060826,0.022933,0.098666,0.47,0.094,8.297872,0.567923,0.4,-0.167923,-1.38,-0.276,-3.405797,0.000000,0.0,0.000000
9370,2024-03-26,China Cba,08:35,Guangzhou,Zhejiang Guangsha,1.75,2.01,197.5,1.83,1.91,-1.5,1.86,2.12,x6XVDs9l,0.571429,0.497512,0.546448,0.523560,0.068941,0.6,0.2,NaN,NaN,193.762,66.251055,0.341920,135.522,14.479189,0.106840,242.694,150.114,181.39,153.00,0.0,0.0,0.0,0.0,0.097791,0.030251,0.092386,3.01,0.602,1.245847,0.536378,0.6,0.063622,-2.88,-0.576,-1.753472,0.616789,0.2,-0.416789
9371,2024-03-26,China Cba,08:35,Shandong,Beijing Royal Fighters,1.86,1.88,217.5,1.81,1.91,-1.5,1.95,1.98,tbUnAa8D,0.537634,0.531915,0.552486,0.523560,0.069549,0.8,0.6,NaN,NaN,280.516,147.938501,0.527380,205.632,68.980176,0.335454,213.868,206.876,113.36,181.50,0.0,0.0,0.0,0.0,0.007563,0.038016,0.010796,-1.08,-0.216,-3.981481,0.614210,0.6,-0.014210,1.07,0.214,4.112150,0.491131,0.5,0.008869
9372,2024-03-26,Coreia Do Sul Kbl,07:00,KoGas,Mobis Phoebus,1.98,1.80,167.5,1.80,1.92,1.5,1.86,1.66,QDM9xaJh,0.505051,0.555556,0.555556,0.520833,0.060606,0.8,1.0,NaN,NaN,152.036,30.513975,0.200702,181.456,25.838810,0.142397,201.994,216.270,158.40,219.78,0.0,0.0,0.0,0.0,0.067344,0.045620,0.080353,2.15,0.430,2.279070,0.530851,0.7,0.169149,2.58,0.516,1.550388,0.495250,0.6,0.104750
9373,2024-03-26,Dinamarca Liga De Basquete,15:00,Holbaek-Stenhus,Herlev Wolfpack,2.01,1.75,166.5,1.86,1.86,1.5,1.87,1.61,4hL9G07U,0.497512,0.571429,0.537634,0.537634,0.068941,0.0,0.6,NaN,NaN,518.784,315.984529,0.609087,550.928,486.270670,0.882639,0.000,631.408,177.84,1174.50,0.0,1.0,0.0,0.0,0.097791,0.000000,0.105660,0.00,0.000,inf,0.000000,0.0,0.000000,-5.00,-1.000,-0.750000,0.000000,0.0,0.000000
9374,2024-03-26,Eua Nit,22:00,Indiana State,Cincinnati,1.64,2.23,154.5,1.80,1.95,-3.5,1.91,2.80,f3BJ9F2F,0.609756,0.448430,0.555556,0.512821,0.058187,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.215604,0.056569,0.267229,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9375,2024-03-26,Finlândia Korisliiga,13:30,Salon Vilpas,Helsinki Seagulls,1.99,1.77,166.5,1.87,1.79,1.5,1.83,1.63,vFcD2PUr,0.502513,0.564972,0.534759,0.558659,0.067484,0.2,0.0,NaN,NaN,145.734,41.598384,0.285440,112.138,34.829547,0.310595,135.892,109.236,106.40,169.15,0.0,0.0,0.0,0.0,0.082747,0.030912,0.081746,-0.56,-0.112,-8.839286,0.000000,0.0,0.000000,-0.95,-0.190,-4.052632,0.000000,0.0,0.000000
9376,2024-03-26,França Pro B,16:00,ASA,Chalons-Reims,1.74,1.99,154.5,1.81,1.85,-2.5,2.01,2.39,YmiiTTHQ,0.574713,0.502513,0.552486,0.540541,0.077225,0.2,0.8,NaN,NaN,155.128,51.270018,0.330501,143.444,53.677875,0.374208,146.096,200.718,146.20,216.81,0.0,0.0,0.0,0.0,0.094786,0.015456,0.122137,2.82,0.564,1.312057,0.000000,0.0,0.

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,21:00,Argentina Liga A,Comunicaciones Mercedes,San Martin,157.5,1.82,1.0000,Over
1,08:35,China Cba,Guangzhou,Zhejiang Guangsha,197.5,1.83,0.9943,Over
2,08:35,China Cba,Shandong,Beijing Royal Fighters,217.5,1.81,0.8343,Over
3,22:00,Eua Nit,Indiana State,Cincinnati,154.5,1.80,1.0000,Over
4,16:00,França Pro B,ASA,Chalons-Reims,154.5,1.81,1.0000,Over
5,16:00,França Pro B,Poitiers,Nantes,165.5,1.87,1.0000,Over
6,14:15,Israel Liga Leumit,Maccabi Maale Adumim,Safed,157.5,1.81,1.0000,Over
7,14:00,Rússia Liga Vtb United,Lokomotiv Kuban,Unics Kazan,165.5,1.89,1.0000,Over
8,23:00,Eua Nba,Sacramento Kings,Dallas Mavericks,236.5,1.86,1.0000,Over
